In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# SILVER LAYER SCRIPT

### DATA ACCESS USING APP

In [0]:
spark.conf.set("fs.azure.account.auth.type.strorageaccaw.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.strorageaccaw.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.strorageaccaw.dfs.core.windows.net", "03a889-18a4-486a-8d16-6d03ad62087e")
spark.conf.set("fs.azure.account.oauth2.client.secret.strorageaccaw.dfs.core.windows.net", "THP8QpirImZA6n7FTKtJW21xOU7V1GcQAJau.")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.strorageaccaw.dfs.core.windows.net", "https://login.microsoftonline.com/f4978633-07e0-4bc6-8cfd-e0f040036/oauth2/token")

# DATA LOADING

## DATA READING

In [0]:
df_cal = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Calendar')

In [0]:
df_cus = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_procat = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [0]:
df_pro = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Products')

In [0]:
df_ret = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Returns')

In [0]:
df_sales = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Sales*')

In [0]:
df_ter = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/AdventureWorks_Territories')

In [0]:
df_subcat = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@strorageaccaw.dfs.core.windows.net/Product_Subcategories')

# TRANSFORMATIONS

## Calender

In [0]:
df_cal.display()

In [0]:
df_cal = df_cal.withColumn('Month',month(col('Date')))\
            .withColumn('Year',year(col('Date')))

In [0]:
display(df_cal)

In [0]:
df_cal.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_Calendar")\
            .save()

## Customers

In [0]:
df_cus.display()

In [0]:
df_cus.withColumn("fullName",concat(col('Prefix'),lit(' '),col('FirstName'),lit(' '),col('LastName'))).display() 

In [0]:
df_cus = df_cus.withColumn('fullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('lastName')))

In [0]:
df_cus.display()

In [0]:
df_cus.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_Customers")\
            .save()

## Sub Categories

In [0]:
df_subcat.display()

In [0]:
df_subcat.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_SUbCategories")\
            .save()

## Products

In [0]:
df_pro.display()

In [0]:
df_pro = df_pro.withColumn('ProductSKU',split(col('ProductSKU'),'-')[0])\
                .withColumn('ProductName',split(col('ProductName'),' ')[0])

In [0]:
df_pro.display()

In [0]:
df_pro.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_Products")\
            .save()

## Returns

In [0]:
display(df_ret)

In [0]:
df_ret.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_Returns")\
            .save()

## Territories

In [0]:
df_ter.display()

In [0]:
df_ter.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_Territories")\
            .save()

## Sales

In [0]:
df_sales.display()

In [0]:
df_sales = df_sales.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_sales = df_sales.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sales = df_sales.withColumn('multiply',col('OrderLineItem')*col('OrderQuantity'))

In [0]:
df_sales.display()

In [0]:
df_sales.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@strorageaccaw.dfs.core.windows.net/AdventureWorks_Sales")\
            .save()

## Sales Analysis

In [0]:
df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order')).display()

In [0]:
df_procat.display()

In [0]:
df_ter.display()